# Background analysis with $\mu p \rightarrow \mu K^0 + \text{others}$

In [1]:
import numpy as np
import ROOT as r

Welcome to JupyROOT 6.26/02


In [2]:
chain=r.TChain("Events")
chain.Add("/Users/isaac/Work/MUonE-displaced-vertex/mup_to_muj/tag_3_pythia8_events.root")

1

## Pick out event particles, especially events that contains 1 final state $K^0$

Here we also need to delete the decay products of $K^0$

In [99]:
event_info_list = [] # particles for the whole events
K0event_info_list = [] # particles for events with 1 final state K0
for event_id, event in enumerate(chain):
    particle_id_list=[]
    particle_status_list = []
    particle_d1_list = [] # daughter 1 of particle, here it refers to the index of the particle list.
    particle_d2_list = [] # daughter 2 of particle
    particle_barcode_list = [] # Particle barcode, the first entry in hepmc. Actually do not need this number for analysis, but I just want it to be output if I would like to manually compare with the hepmc file.
    particle_E_list = []
    particle_px_list = []
    particle_py_list = []
    particle_pz_list = []
    particle_mass_list = []
    for id in event.Particle_pid:
        particle_id_list.append(id)
    for status in event.Particle_status:
        particle_status_list.append(status)
    for d1 in event.Particle_d1:
        particle_d1_list.append(d1)
    for d2 in event.Particle_d2:
        particle_d2_list.append(d2)
    for barcode in event.Particle_barcode:
        particle_barcode_list.append(barcode)
    for energy in event.Particle_energy:
        particle_E_list.append(energy)
    for px in event.Particle_px:
        particle_px_list.append(px)
    for py in event.Particle_py:
        particle_py_list.append(py)
    for pz in event.Particle_pz:
        particle_pz_list.append(pz)
    for mass in event.Particle_mass:
        particle_mass_list.append(mass)

    event_particles = np.array([particle_barcode_list,particle_id_list,particle_status_list, particle_d1_list, particle_d2_list, particle_E_list, particle_px_list, particle_py_list, particle_pz_list, particle_mass_list]).transpose().tolist()
    event_info_list.append(event_particles)
    K0_index_list=[]
    for index, pid in enumerate(particle_id_list):
        if pid==130:
            K0_index_list.append(index)
    K0_status_list = [particle_status_list[x] for x in K0_index_list]
    if K0_status_list.count(1)==1:
        K0event_particles = np.array([particle_barcode_list,particle_id_list,particle_status_list, particle_d1_list, particle_d2_list,particle_E_list, particle_px_list, particle_py_list, particle_pz_list, particle_mass_list]).transpose().tolist()
        #for index, particle in enumerate(K0event_particles):
        #    if particle[1] == 130 and particle[2]==2:
        #        daughter_index = [particle[3],particle[4]]
        #real_daughter = [i for i in daughter_index if i!=-1]
        #for daughters in real_daughter:
        #    del K0event_particles[int(daughters)]
        K0event_info_list.append(K0event_particles)

In [100]:
len(event_info_list)

9998

In [101]:
len(K0event_info_list)

887

In [102]:
K0event_info_list[0]

[[1.0,
  13.0,
  4.0,
  0.0,
  -1.0,
  160.00846151213833,
  -2.220446049250313e-16,
  -2.6645352591003757e-15,
  160.00842662636816,
  0.10566],
 [4.0,
  2.0,
  21.0,
  2.0,
  3.0,
  0.12841834616234285,
  -8.326672684688674e-17,
  3.3306690738754696e-16,
  -0.12841834616234182,
  0.0],
 [5.0,
  13.0,
  1.0,
  -1.0,
  -1.0,
  113.46764362527337,
  0.8633275049599003,
  -4.0257494880502875,
  113.3928701132266,
  0.10566],
 [6.0,
  2.0,
  23.0,
  6.0,
  -1.0,
  46.64628042045924,
  -0.8633275049599007,
  4.025749488050287,
  46.4642172401813,
  0.0],
 [2.0,
  2212.0,
  4.0,
  4.0,
  5.0,
  0.9382700013119802,
  -3.3306690738754696e-16,
  1.3322676295501878e-15,
  4.961859254493106e-05,
  0.93827],
 [7.0,
  2.0,
  61.0,
  1.0,
  -1.0,
  4.890875954632406,
  -0.12155074237697647,
  1.9562679318566167,
  4.480949542163858,
  0.0],
 [8.0,
  2.0,
  62.0,
  7.0,
  13.0,
  44.87570681703754,
  -0.8628373408131025,
  5.389727434069277,
  44.542512414977764,
  0.0],
 [10.0,
  211.0,
  1.0,
  -1

## Require number of final state particles
- The final state particle muon should count.
- Beyond muon and K0, there should be only one final state particle. Do not care for now whether it carry positive or negative charge.

In [103]:
pass_event_info_list = []
for event in K0event_info_list:
    final_state_pass_cut = []
    for particle in event:
        if particle[2] == 1 and particle[1]!=13:
            energy = particle[5]
            px = particle[6]
            py = particle[7]
            pz = particle[8]
            mass = particle[9]
            tantheta = np.arctan(np.sqrt(px**2 + py**2)/pz)
            if energy >= 5 and tantheta <= 0.05:
                final_state_pass_cut.append(particle)
    if len(final_state_pass_cut) == 1:
        pass_event_info_list.append(event)

In [104]:
len(pass_event_info_list)

147

In [144]:
pass_event_info_list[0]

[[1.0,
  13.0,
  4.0,
  0.0,
  -1.0,
  160.00846151175912,
  1.1102230246251565e-15,
  -2.6645352591003757e-15,
  160.00842662598902,
  0.10566],
 [4.0,
  2.0,
  21.0,
  2.0,
  3.0,
  0.07140986756000786,
  1.3877787807814457e-17,
  -1.6653345369377348e-16,
  -0.071409867560001,
  0.0],
 [5.0,
  13.0,
  1.0,
  -1.0,
  -1.0,
  110.31067283859934,
  -1.0343143872966958,
  2.9517057762172243,
  110.266273197328,
  0.10566],
 [6.0,
  2.0,
  23.0,
  4.0,
  5.0,
  49.7462427289121,
  1.0343143872966962,
  -2.9517057762172243,
  49.64782263506343,
  0.0],
 [7.0,
  2.0,
  51.0,
  9.0,
  -1.0,
  34.85986022198622,
  0.1568267137985176,
  -2.1144127745333625,
  34.79532323024383,
  0.0],
 [8.0,
  21.0,
  51.0,
  10.0,
  -1.0,
  14.894173630627886,
  0.8774876734981787,
  -0.8372930016838612,
  14.844708281117587,
  0.0],
 [9.0,
  2.0,
  53.0,
  1.0,
  -1.0,
  0.07920099126201141,
  1.942890293094024e-16,
  -5.551115123125783e-17,
  -0.07920099126201063,
  0.0],
 [2.0,
  2212.0,
  4.0,
  7.0,
  8

## Now add cuts to K0 and muon
- Both should be within the angular acceptance
- Least energy cuts: muon is 5, K0 is 10 since K0 should have its decayed pion to have more than 5.
- Opening angle cut

In [145]:
second = 3e08
m_K0 = 0.497611
K0_life_time = 5.116e-08*second
branching_to_pionpair = 1.967e-03
# Use as length unit
l1 = 0.02
l2 = 0.145
event_number = 0
K0p_list = []
for event in pass_event_info_list:
    for particle in event:
        if particle[2]==1 and particle[1]==13:
            muon_energy = particle[5]
            px = particle[6]
            py = particle[7]
            pz = particle[8]
            mass = particle[9]
            muon_tantheta = np.arctan(np.sqrt(px**2 + py**2)/pz)
        if particle[2]==1 and particle[1]==130:
            K0_energy = particle[5]
            K0_px = particle[6]
            K0_py = particle[7]
            K0_pz = particle[8]
            K0_tantheta = np.arctan(np.sqrt(px**2 + py**2)/pz)
    K0p_list.append(np.sqrt(K0_px**2 + K0_py**2 + K0_pz**2))
    opening_angle = 2*m_K0/K0_energy
    decay_distance = np.sqrt(K0_px**2 + K0_py**2 + K0_pz**2)*K0_life_time/m_K0
    prob = np.exp(-l1/decay_distance)*(1-np.exp(-(l2-l1)/decay_distance))
    if muon_energy >= 5 and muon_tantheta <= 0.05 and K0_energy >= 10 and K0_tantheta <= 0.05 and opening_angle >= 0.001:
        event_number += prob

In [146]:
event_number/9998*3e04*1.5e04*branching_to_pionpair

0.7582615464731489

In [142]:
second = 3e08
m_K0 = 0.497611
K0_life_time = 5.116e-08*second
branching_to_pionpair = 1.967e-03
# Use as length unit
l1 = 0.025
l2 = 0.14
event_number = 0
K0p_list = []
for event in pass_event_info_list:
    for particle in event:
        if particle[2]==1 and particle[1]==13:
            muon_energy = particle[5]
            px = particle[6]
            py = particle[7]
            pz = particle[8]
            mass = particle[9]
            muon_tantheta = np.arctan(np.sqrt(px**2 + py**2)/pz)
        if particle[2]==1 and particle[1]==130:
            K0_energy = particle[5]
            K0_px = particle[6]
            K0_py = particle[7]
            K0_pz = particle[8]
            K0_tantheta = np.arctan(np.sqrt(px**2 + py**2)/pz)
    K0p_list.append(np.sqrt(K0_px**2 + K0_py**2 + K0_pz**2))
    opening_angle = 2*m_K0/K0_energy
    decay_distance = np.sqrt(K0_px**2 + K0_py**2 + K0_pz**2)*K0_life_time/m_K0
    prob = np.exp(-l1/decay_distance)*(1-np.exp(-(l2-l1)/decay_distance))
    if muon_energy >= 5 and muon_tantheta <= 0.05 and K0_energy >= 10 and K0_tantheta <= 0.05 and opening_angle >= 0.001:
        event_number += prob

In [143]:
event_number/9998*3e04*1.5e04*branching_to_pionpair

0.6976006224044409

In [131]:
from matplotlib import pyplot as plt

In [134]:
paverage=sum(K0p_list)/len(K0p_list)
print(paverage)

In [135]:
paverage*K0_life_time/m_K0

8.726786878218289e-07

In [116]:
event_number = 0
for event in pass_event_info_list:    
    for particle in event:
        if particle[2]==1 and particle[1]==13:
            muon_energy = particle[5]
            px = particle[6]
            py = particle[7]
            pz = particle[8]
            mass = particle[9]
            muon_tantheta = np.arctan(np.sqrt(px**2 + py**2)/pz)
        if particle[2]==1 and particle[1]==130:
            K0_energy = particle[5]
            K0_px = particle[6]
            K0_py = particle[7]
            K0_pz = particle[8]
            K0_tantheta = np.arctan(np.sqrt(px**2 + py**2)/pz)

In [121]:
dbar = 1000*np.sqrt(K0_px**2 + K0_py**2 + K0_pz**2)*K0_life_time/m_K0

In [122]:
np.exp(125/dbar)

/var/folders/f3/5638tcs57qd4mv3__k9pyl140000gn/T/ipykernel_36153/1485693572.py:1: RuntimeWarning: overflow encountered in exp
  np.exp(125/dbar)


inf

In [123]:
second = 3e08

913465.7382193677

In [127]:
10/decay_rate(0.01,1.0e-4)/0.01

0.07427029940763681